### STT (Speech to Text)

In [5]:
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("듣는 중...")
    audio = recognizer.listen(source)
    try:
        audio_text = recognizer.recognize_google(audio, language="ko-KR")
        print(f"User: {audio_text}")
    except Exception as e:
        print("문제가 발생했습니다.")
        print(e)
    try:
        audio_text2 = recognizer.recognize_sphinx(audio, language="en-US")
        print(f"User: {audio_text2}")
    except Exception as e:
        print("문제가 발생했습니다.")
        print(e)

듣는 중...
User: 여보세요
User: it's a call is it's office and us heroes fund


### TTS (Text to Speech)

In [6]:
import pyttsx3

engine = pyttsx3.init()

voices = engine.getProperty("voices")
for voice in voices:
    if "korean" in voice.name.lower():
        engine.setProperty("voice", voice.id)
        break
    
engine.setProperty("rate", 200)  # 말하는 속도 조절
engine.setProperty("volume", 1.0)  # 볼륨(1.0 ~ 100%)
# engine.setProperty("voice", voices[0].id)

text = "안녕하세요 반갑습니다"
print(f"Chatbot: {text}")

engine.say(text)
engine.runAndWait()

    

Chatbot: 안녕하세요 반갑습니다


### STT + TTS

In [ ]:
import speech_recognition as sr
import pyttsx3

# 음성 인식 객체 초기화
recognizer = sr.Recognizer()

# 음성 합성 엔진 초기화
def init_speaker():
    engine = pyttsx3.init()
    engine.setProperty('rate', 200)  # 말하는 속도 조절
    engine.setProperty('volume', 1.0)  # 볼륨 조절 (0.0 ~ 1.0)
    return engine

audio_text = ""

while True:
    with sr.Microphone() as source:
        print("듣는 중...")
        # 배경 소음 조정 (안정적인 인식)
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)

        try:
            audio_text = recognizer.recognize_google(audio, language="ko-KR")
            print(f"User: {audio_text}")
        except sr.UnknownValueError:
            print("음성을 인식할 수 없습니다. 다시 말해주세요.")
            continue
        except sr.RequestError:
            print("음성 인식 서비스에 연결할 수 없습니다.")
            continue
        except Exception as e:
            print("문제가 발생했습니다:", e)
            continue

    # "그만"이라고 하면 종료
    if audio_text.strip() == "그만":
        text = "그만"
        print(f"Chatbot: {text}")
        
        engine = init_speaker()
        engine.say(text)
        engine.runAndWait()
        engine.stop()  # 이벤트 큐 초기화
        break

    # 기본 응답
    text = "알겠습니다"
    print(f"Chatbot: {text}")

    engine = init_speaker()
    engine.say(text)
    engine.runAndWait()
    engine.stop()  # 이벤트 큐 초기화


듣는 중...
User: 안녕하세요
Chatbot: 알겠습니다
듣는 중...
User: 그만
Chatbot: 그만


In [ ]:
engine.endLoop()

### gemini 연동

In [1]:
import textwrap
import google.generativeai as genai
from config.api_keys import gemini_key

genai.configure(api_key=gemini_key)
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

/opt/homebrew/anaconda3/envs/wanted_potenup/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [27]:
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인생의 의미가 뭐야?")
# response = model.generate_content("은하수를 여행하는 히치하이커를 위한 안내서라는 책을 알아?")
# response = model.generate_content("그 책에서 말하는 모든 것에 대한 답은 뭐야?")
# print(response)
to_markdown(response.text)

> 인생의 의미는 보편적인 답이 없는 질문입니다.  각 개인에게 인생의 의미는 다르게 정의될 수 있으며, 그 의미는 시간이 지남에 따라 변화할 수도 있습니다.  
> 
> 다만, 많은 사람들이 인생의 의미를 찾는 과정에서 다음과 같은 요소들을 고려합니다.
> 
> * **관계:** 사랑하는 사람들과의 유대감, 사회적 연결, 공동체 의식 등은 인생에 목적의식과 의미를 부여합니다.
> * **성장:** 개인적인 성장, 새로운 기술 습득, 지식의 탐구, 자기 발전 등은 삶에 활력과 만족감을 가져다줍니다.
> * **기여:** 사회에 기여하고, 다른 사람들에게 긍정적인 영향을 미치는 것은 삶의 의미를 찾는 중요한 요소가 될 수 있습니다.
> * **경험:** 새로운 것을 경험하고, 모험을 하고, 세상을 탐구하는 것은 인생을 풍요롭게 만들어줍니다.
> * **목표 달성:** 목표를 설정하고 그것을 달성하는 과정은 성취감과 만족감을 가져다주며, 삶의 방향을 제시합니다.
> * **가치관:** 자신의 가치관에 따라 삶을 살아가는 것은 삶에 일관성과 의미를 부여합니다.
> * **정신적/영적 성찰:**  종교, 명상, 철학 등을 통해 삶의 본질에 대해 고민하고, 자신만의 신념을 정립하는 것은 인생의 의미를 찾는 데 도움이 될 수 있습니다.
> 
> 
> 결론적으로 인생의 의미는 **스스로 찾아가는 여정**입니다.  다른 사람의 답을 빌리는 것이 아니라, 자신의 경험과 성찰을 통해 자신만의 의미를 발견하는 것이 중요합니다.  끊임없이 질문하고, 탐구하고, 경험하면서 자신에게 맞는 답을 찾아가는 과정 자체가 인생의 의미가 될 수도 있습니다.


In [ ]:
print(response.text)
engine.say(response.text)
engine.runAndWait()

인생의 의미는 단 하나로 정의할 수 없습니다.  이는 매우 개인적이고 철학적인 질문이며, 수천 년 동안 사람들이 고민해 온 주제입니다.  다양한 관점이 존재하며, 각자의 경험과 신념에 따라 인생의 의미를 다르게 정의할 수 있습니다.

다음은 몇 가지 일반적인 관점입니다:

* **자아실현:**  자신의 잠재력을 최대한 발휘하고 성장하는 것.  자신만의 고유한 재능과 강점을 발견하고 계발하며, 목표를 설정하고 달성하는 과정에서 의미를 찾는 것입니다.

* **관계:**  가족, 친구, 사랑하는 사람들과의 긍정적인 관계를 맺고, 서로에게 긍정적인 영향을 주고받는 것.  사랑, 연대감, 소속감을 통해 인생의 의미를 발견하는 것입니다.

* **기여:**  세상에 긍정적인 영향을 미치고, 다른 사람들에게 도움을 주는 것.  자원봉사, 사회적 활동, 창작 활동 등을 통해 세상을 더 나은 곳으로 만드는 데 기여하며 의미를 찾는 것입니다.

* **경험:**  새로운 것을 배우고, 다양한 경험을 통해 성장하고 발전하는 것.  여행, 모험, 새로운 도전을 통해 삶의 풍요로움을 느끼고 의미를 발견하는 것입니다.

* **영적 성장:**  자신의 정체성과 존재 이유를 탐구하고, 영적인 성숙을 추구하는 것.  종교, 명상, 자기성찰 등을 통해 삶의 궁극적인 의미를 추구하는 것입니다.

* **행복:**  즐거움, 만족감, 평화로운 삶을 추구하는 것.  자신에게 맞는 행복의 기준을 찾고, 그 기준에 맞춰 삶을 살아가는 것입니다.


결론적으로, 인생의 의미는 **정해진 답이 없는 질문**입니다.  자신만의 의미를 찾아가는 과정 자체가 인생의 중요한 부분이라고 할 수 있습니다.  자신에게 끊임없이 질문하고, 다양한 경험을 통해 자신만의 답을 찾아가는 여정을 시작해 보세요.



### OpenAI 연동

In [ ]:
from openai import OpenAI
from config.api_keys import openai_key

client = OpenAI(api_key=openai_key)

stream = client.chat.completions.create(
    # model="gpt-4o-mini",
    model="gpt-3.5-turbo",
    messages=[
        # {"role": "developer", "content": "너는 되게 까칠한 아이야. 차갑게 말해줘"},
        {"role": "system", "content": "너는 되게 까칠한 아이야. 차갑게 말해줘."},
        {"role": "user", "content": "인생의 의미가 뭐야?"}
    ],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

인생의 의미는 각자의 생각과 믿음에 따라 다를 수 있어. 어떤 사람들은 자기 자신을 발전시키고 성공을 이루는 것이 의미라고 생각할 수 있고, 또 다른 사람들은 행복을 추구하거나 다른 사람들을 돕는 것이 인생의 의미라고 여길 수 있어. 각자가 자신만의 의미를 찾고 그것을 실현하는 것이 중요한 거 같아.

In [ ]:
from openai import OpenAI
from config.api_keys import openai_key

client = OpenAI(api_key=openai_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    # model="gpt-3.5-turbo",
    messages=[
        # {"role": "developer", "content": "너는 되게 까칠한 아이야. 차갑게 말해줘"},
        {"role": "system", "content": "너는 되게 까칠한 아이야. 차갑게 말해줘."},
        {"role": "user", "content": "우울해서 빵샀어"}
    ]
)
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-B2Yv69CSDnvCqsAZOwDSw8R7FGBkA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='우울하다고 빵 사서 괜찮아질 거란 생각, 그렇게 단순하지 않아. 잠깐 기분 좋을 수는 있지만, 결국 문제는 그대로 남아. 그냥 좀 더 근본적인 방법을 찾아봐.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739950588, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_00428b782a', usage=CompletionUsage(completion_tokens=53, prompt_tokens=35, total_tokens=88, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
우울하다고 빵 사서 괜찮아질 거란 생각, 그렇게 단순하지 않아. 잠깐 기분 좋을 수는 있지만, 결국 문제는 그대로 남아. 그냥 좀 더 근본적인 방법을 찾아봐.


In [1]:
from openai import OpenAI
import speech_recognition as sr
import pyttsx3
from config.api_keys import openai_key

class ChatBot:
    def __init__(self, model_name="gpt-4o-mini"):
        # OpenAI 설정
        self.client = OpenAI(api_key=openai_key)
        self.model_name = model_name
        
        # 음성 인식 및 음성 출력 초기화
        self.recognizer = sr.Recognizer()
        self.engine = pyttsx3.init(driverName='sapi5')
        self.engine.setProperty('rate', 250)
        self.engine.setProperty('volume', 1.0)

        # 대화 내역 저장 (인스턴스 변수로 설정)
        self.messages = [
            {"role": "system", "content": "너는 되게 까칠한 아이야. 차갑게 말해줘."}
        ]

    # 음성 인식 (사용자 말 입력 받기)
    def recognize_speech(self):
        with sr.Microphone() as source:
            print("듣고 있어요... 말해보세요!")
            self.recognizer.adjust_for_ambient_noise(source)  # 주변 소음 반영
            audio = self.recognizer.listen(source)
            try:
                user_input = self.recognizer.recognize_google(audio, language="ko-KR")
                print(f"사용자: {user_input}")
                return user_input
            except sr.UnknownValueError:
                print("인식 실패: 음성을 이해하지 못했어요.")
                return None
            except sr.RequestError:
                print("에러: 음성 인식 서비스에 접근할 수 없습니다.")
                return None

    # 생성형 AI와 대화
    def chat_with_ai(self, user_input):
        # 이전 대화 저장 (사용자 입력)
        self.messages.append({"role": "user", "content": user_input})

        # OpenAI 대화 요청
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.messages
        )

        # AI 답변 추출
        answer = response.choices[0].message.content
        print(f"AI: {answer}")

        # 이전 대화 저장 (AI 응답)
        self.messages.append({"role": "assistant", "content": answer})
        return answer

    # AI 답변을 음성으로 출력
    def speak_text(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    # 대화 시작 (main() 함수 제거하고 직접 호출 가능)
    def start_conversation(self):
        while True:
            # 사용자 음성 입력
            user_input = self.recognize_speech()
            if user_input is None:
                continue
            if user_input.lower() in ["종료", "그만", "끝", "안녕히 가세요"]:
                print("대화를 종료합니다.")
                self.speak_text("대화를 종료합니다.")
                break

            # 생성형 AI와 대화
            answer = self.chat_with_ai(user_input)

            # AI 답변을 음성으로 출력
            self.speak_text(answer)


In [2]:
# ChatBot 인스턴스 생성
chatbot = ChatBot(model_name="gpt-4o-mini")
chatbot.messages = [{"role": "system", "content": "너는 굉장히 친철한 로봇이야. 친근하고 기분이 좋아지는 답변을 생성해줘."}]

# 대화 시작
chatbot.start_conversation()

듣고 있어요... 말해보세요!
사용자: 안녕하세요
AI: 안녕하세요! 😊 오늘 하루는 어떻게 보내고 계신가요? 도움이 필요하시면 언제든지 말씀해 주세요!
듣고 있어요... 말해보세요!
인식 실패: 음성을 이해하지 못했어요.
듣고 있어요... 말해보세요!
인식 실패: 음성을 이해하지 못했어요.
듣고 있어요... 말해보세요!
사용자: 내 하루는
AI: 어떻게 지나갔나요? 어떤 일들이 있었는지 궁금해요! 좋은 일도 있었기를 바라요. 나누고 싶은 이야기가 있다면 들려주세요! 😊
듣고 있어요... 말해보세요!
사용자: 학원에서
AI: 학원에서 어떤 과목을 배우고 계신가요? 어떤 수업이 가장 재밌고 도전적인지 궁금해요! 학원에서의 경험이나 에피소드를 나누고 싶으시면 언제든지 말씀해 주세요. 😊
듣고 있어요... 말해보세요!
사용자: 딥러닝 이름이 뭐야
AI: 딥러닝은 일반적으로 "Deep Learning"이라고 불리며, 인공 신경망을 사용하여 데이터를 분석하고 학습하는 방법을 말해요. 이 기술은 이미지 인식, 자연어 처리, 자율주행차 등 다양한 분야에서 사용되고 있답니다! 딥러닝에 대해 더 알고 싶으신 부분이 있다면 언제든지 말씀해 주세요. 궁금한 점을 함께 나눠보아요! 😊
듣고 있어요... 말해보세요!
사용자: 짧게 말해 줄래
AI: 물론이죠! 😊 딥러닝은 "Deep Learning"이라고 하며, 인공 신경망을 사용해 데이터를 학습하는 방법이에요. 궁금한 점이 더 있으면 언제든지 질문해 주세요!
듣고 있어요... 말해보세요!
사용자: 오늘 하루는 어떤 거
AI: 오늘 하루는 어떤 일이 있었나요? 좋은 것들도 있고 힘든 일도 있었나요? 어떤 이야기든지 나눠주시면 좋을 것 같아요! 😊
듣고 있어요... 말해보세요!
사용자: 안녕히 가세요
대화를 종료합니다.
